In [1]:
import keras
from keras import layers
import tensorflow as tf
from keras import applications, models, layers

import os
print(os.listdir('./data'))


train_dataset = keras.utils.image_dataset_from_directory(
    './data',                       
    labels='inferred',             
    label_mode='int',              
    batch_size=32,                 
    image_size=(256, 256),         
    shuffle=True,                  
    validation_split=0.5,          
    subset='training',             
    seed=123                       
)

validation_dataset = keras.utils.image_dataset_from_directory(
    './data',                    
    labels='inferred',           
    label_mode='int',            
    batch_size=32,               
    image_size=(256, 256),       
    shuffle=True,                
    validation_split=0.2,        
    subset='validation',         
    seed=123                     
)

# normalize pixel values to [0, 1]
normalization_layer = layers.Rescaling(1./255)

train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
validation_dataset = validation_dataset.map(lambda x, y: (normalization_layer(x), y))

# Prefetch to optimize performance during training
# AUTOTUNE = tf.data.AUTOTUNE
# train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
# validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

base_model = applications.MobileNetV2(input_shape=(256, 256, 3), include_top=False, weights='imagenet')
base_model.trainable = False  

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Assuming binary classification (fire risk vs no fire risk)
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# training
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=10
)

# unfreeze base model layers and fine tune
base_model.trainable = True
for layer in base_model.layers[:-40]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

fine_tune_history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=5
)

model.save('fire_risk_model.h5')

['fire', 'nofire']
Found 3584 files belonging to 2 classes.
Using 1792 files for training.
Found 3584 files belonging to 2 classes.
Using 716 files for validation.


/var/folders/4s/0shkycv9613d_rw6h0lfrjqw0000gn/T/ipykernel_9522/2035116742.py:45: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = applications.MobileNetV2(input_shape=(256, 256, 3), include_top=False, weights='imagenet')


Epoch 1/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 22s 369ms/step - accuracy: 0.8575 - loss: 0.2603 - val_accuracy: 0.9623 - val_loss: 0.1052
Epoch 2/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 21s 374ms/step - accuracy: 0.9780 - loss: 0.0511 - val_accuracy: 0.9763 - val_loss: 0.0717
Epoch 3/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 21s 377ms/step - accuracy: 0.9907 - loss: 0.0289 - val_accuracy: 0.9804 - val_loss: 0.0573
Epoch 4/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 20s 351ms/step - accuracy: 0.9962 - loss: 0.0171 - val_accuracy: 0.9818 - val_loss: 0.0566
Epoch 5/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 20s 352ms/step - accuracy: 0.9986 - loss: 0.0116 - val_accuracy: 0.9804 - val_loss: 0.0619
Epoch 6/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 21s 372ms/step - accuracy: 0.9974 - loss: 0.0089 - val_accuracy: 0.9832 - val_loss: 0.0603
Epoch 7/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 22s 386ms/step - accuracy: 0.9998 - loss: 0.0071 - val_accuracy: 0.9832 - val_loss: 0.0592
Epoch 8/10
56/56 ━━━━━━━━━━━━━━━━━━━━ 22s 396ms/step - accuracy: 0.9995 - loss: 0.0046 - val_accu

In [3]:
# Load testing dataset
test_dataset = keras.utils.image_dataset_from_directory(
    './testing',                    # Path to testing data
    labels='inferred',              # Infer labels from directory structure
    label_mode='int',               # Integer labels for binary classification
    batch_size=32,                  # Use same batch size as for training
    image_size=(256, 256),          # Use same image size as for training
    shuffle=False                   # No need to shuffle the testing data
)

# Normalize the pixel values of the testing dataset
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))

# Evaluate the model on the testing data
test_loss, test_accuracy = model.evaluate(test_dataset)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Found 40 files belonging to 2 classes.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 1.0000 - loss: 0.0022
Test Loss: 0.002107433509081602
Test Accuracy: 1.0
